In [9]:
import pandas as pd
from scipy import sparse
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
ratings = pd.read_csv('ratings.csv')
movies = pd.read_csv('movies.csv')
posters = pd.read_csv('movie_posters.csv',header=None,names=['movieId','poster_url'],index_col=0)
ratings = pd.merge(movies,ratings).drop(['genres','timestamp'],axis=1)
ratings.head()

movieId             title  userId  rating
0        1  Toy Story (1995)       1     4.0
1        1  Toy Story (1995)       5     4.0
2        1  Toy Story (1995)       7     4.5
3        1  Toy Story (1995)      15     2.5
4        1  Toy Story (1995)      17     4.5

In [11]:
user_ratings = ratings.pivot_table(index=['userId'],columns=['title'],values='rating')
user_ratings.head()

title   '71 (2014)  'Hellboy': The Seeds of Creation (2004)  \
userId                                                        
1              NaN                                      NaN   
2              NaN                                      NaN   
3              NaN                                      NaN   
4              NaN                                      NaN   
5              NaN                                      NaN   

title   'Round Midnight (1986)  'Salem's Lot (2004)  \
userId                                                
1                          NaN                  NaN   
2                          NaN                  NaN   
3                          NaN                  NaN   
4                          NaN                  NaN   
5                          NaN                  NaN   

title   'Til There Was You (1997)  'Tis the Season for Love (2015)  \
userId                                                               
1                             NaN                              NaN   
2                             NaN                              NaN   
3                             NaN                              NaN   
4                             NaN                              NaN   
5                             NaN                              NaN   

title   'burbs, The (1989)  'night Mother (1986)  (500) Days of Summer (2009)  \
userId                                                                          
1                      NaN                   NaN                          NaN   
2                      NaN                   NaN                          NaN   
3                      NaN                   NaN                          NaN   
4                      NaN                   NaN                          NaN   
5                      NaN                   NaN                          NaN   

title   *batteries not included (1987)  \
userId                                   
1                                  NaN   
2                                  NaN   
3                                  NaN   
4                                  NaN   
5                                  NaN   

title                     ...                      Zulu (2013)  [REC] (2007)  \
userId                    ...                                                  
1                         ...                              NaN           NaN   
2                         ...                              NaN           NaN   
3                         ...                              NaN           NaN   
4                         ...                              NaN           NaN   
5                         ...                              NaN           NaN   

title   [REC]² (2009)  [REC]³ 3 Génesis (2012)  \
userId                                           
1                 NaN                      NaN   
2                 NaN                      NaN   
3                 NaN                      NaN   
4                 NaN                      NaN   
5                 NaN                      NaN   

title   anohana: The Flower We Saw That Day - The Movie (2013)  \
userId                                                           
1                                                     NaN        
2                                                     NaN        
3                                                     NaN        
4                                                     NaN        
5                                                     NaN        

title   eXistenZ (1999)  xXx (2002)  xXx: State of the Union (2005)  \
userId                                                                
1                   NaN         NaN                             NaN   
2                   NaN         NaN                             NaN   
3                   NaN         NaN                             NaN   
4                   NaN         NaN                             NaN   
5                

In [12]:
## Let's Remove Movies which have less than 7 users who rated it. and fill remaining NaN with 0

user_ratings = user_ratings.dropna(thresh=7,axis=1).fillna(0)
user_ratings

title   'burbs, The (1989)  (500) Days of Summer (2009)  \
userId                                                    
1                      0.0                          0.0   
2                      0.0                          0.0   
3                      0.0                          0.0   
4                      0.0                          0.0   
5                      0.0                          0.0   
6                      0.0                          0.0   
7                      0.0                          0.0   
8                      0.0                          0.0   
9                      0.0                          0.0   
10                     0.0                          0.0   
11                     0.0                          0.0   
12                     5.0                          0.0   
13                     0.0                          0.0   
14                     0.0                          0.0   
15                     0.0                          4.0   
16                     0.0                          0.0   
17                     0.0                          0.0   
18                     0.0                          4.0   
19                     2.0                          0.0   
20                     0.0                          0.0   
21                     0.0                          0.0   
22                     0.0                          0.5   
23                     0.0                          0.0   
24                     0.0                          0.0   
25                     0.0                          0.0   
26                     0.0                          0.0   
27                     0.0                          0.0   
28                     0.0                          0.0   
29                     0.0                          0.0   
30                     0.0                          0.0   
...                    ...                          ...   
581                    0.0                          0.0   
582                    0.0                          0.0   
583                    0.0                          0.0   
584                    0.0                          0.0   
585                    0.0                          0.0   
586                    0.0                          0.0   
587                    0.0                          0.0   
588                    0.0                          0.0   
589                    0.0                          0.0   
590                    0.0                          0.0   
591                    0.0                          0.0   
592                    0.0                          0.0   
593                    0.0                          0.0   
594                    0.0                          0.0   
595                    0.0                          0.0   
596                    0.0                          0.0   
597                    0.0                          0.0   
598                    0.0                          0.0   
599                    2.0                          2.5   
600                    0.0                          0.0   
601                    0.0                          0.0   
602                    0.0                          0.0   
603                    0.0                          0.0   
604                    0.0                          0.0   
605                    0.0                          0.0   
606                    0.0                          0.0   
607                    0.0                          0.0   
608                    0.0                          0.0   
609                    0.0                          0.0   
610                    0.0                          3.5   

title   *batteries not included (1987)  10 Cloverfield Lane (2016)  \
userId                                                               
1                                  0.0                         0.0   
2                                  0.0                         0.0   
3 

In [13]:
def standardize(row):
    new_row=(row-row.mean())/(row.max()-row.min())
    return new_row
ratings_standardize=user_ratings.apply(standardize)
ratings_standardize

title   'burbs, The (1989)  (500) Days of Summer (2009)  \
userId                                                    
1                -0.017705                    -0.050492   
2                -0.017705                    -0.050492   
3                -0.017705                    -0.050492   
4                -0.017705                    -0.050492   
5                -0.017705                    -0.050492   
6                -0.017705                    -0.050492   
7                -0.017705                    -0.050492   
8                -0.017705                    -0.050492   
9                -0.017705                    -0.050492   
10               -0.017705                    -0.050492   
11               -0.017705                    -0.050492   
12                0.982295                    -0.050492   
13               -0.017705                    -0.050492   
14               -0.017705                    -0.050492   
15               -0.017705                     0.749508   
16               -0.017705                    -0.050492   
17               -0.017705                    -0.050492   
18               -0.017705                     0.749508   
19                0.382295                    -0.050492   
20               -0.017705                    -0.050492   
21               -0.017705                    -0.050492   
22               -0.017705                     0.049508   
23               -0.017705                    -0.050492   
24               -0.017705                    -0.050492   
25               -0.017705                    -0.050492   
26               -0.017705                    -0.050492   
27               -0.017705                    -0.050492   
28               -0.017705                    -0.050492   
29               -0.017705                    -0.050492   
30               -0.017705                    -0.050492   
...                    ...                          ...   
581              -0.017705                    -0.050492   
582              -0.017705                    -0.050492   
583              -0.017705                    -0.050492   
584              -0.017705                    -0.050492   
585              -0.017705                    -0.050492   
586              -0.017705                    -0.050492   
587              -0.017705                    -0.050492   
588              -0.017705                    -0.050492   
589              -0.017705                    -0.050492   
590              -0.017705                    -0.050492   
591              -0.017705                    -0.050492   
592              -0.017705                    -0.050492   
593              -0.017705                    -0.050492   
594              -0.017705                    -0.050492   
595              -0.017705                    -0.050492   
596              -0.017705                    -0.050492   
597              -0.017705                    -0.050492   
598              -0.017705                    -0.050492   
599               0.382295                     0.449508   
600              -0.017705                    -0.050492   
601              -0.017705                    -0.050492   
602              -0.017705                    -0.050492   
603              -0.017705                    -0.050492   
604              -0.017705                    -0.050492   
605              -0.017705                    -0.050492   
606              -0.017705                    -0.050492   
607              -0.017705                    -0.050492   
608              -0.017705                    -0.050492   
609              -0.017705                    -0.050492   
610              -0.017705                     0.649508   

title   *batteries not included (1987)  10 Cloverfield Lane (2016)  \
userId                                                               
1                            -0.009426                   -0.016885   
2                            -0.009426                   -0.016885   
3 

In [14]:
## Let's Build our Similarity Matrix
item_similarity_df = user_ratings.corr(method='pearson')
item_similarity_df.head(50) 

title                                               'burbs, The (1989)  \
title                                                                    
'burbs, The (1989)                                            1.000000   
(500) Days of Summer (2009)                                   0.063117   
*batteries not included (1987)                                0.235908   
10 Cloverfield Lane (2016)                                   -0.023768   
10 Things I Hate About You (1999)                             0.143482   
10,000 BC (2008)                                              0.011998   
101 Dalmatians (1996)                                         0.087931   
101 Dalmatians (One Hundred and One Dalmatians)...            0.224052   
102 Dalmatians (2000)                                        -0.018608   
12 Angry Men (1957)                                           0.034223   
12 Years a Slave (2013)                                       0.009277   
127 Hours (2010)                                              0.008331   
13 Going on 30 (2004)                                         0.049700   
13th Warrior, The (1999)                                      0.176071   
1408 (2007)                                                   0.131993   
15 Minutes (2001)                                             0.104500   
16 Blocks (2006)                                              0.064842   
17 Again (2009)                                               0.047271   
1984 (Nineteen Eighty-Four) (1984)                            0.184452   
2 Days in the Valley (1996)                                   0.092824   
2 Fast 2 Furious (Fast and the Furious 2, The) ...            0.084066   
20 Dates (1998)                                               0.110264   
20,000 Leagues Under the Sea (1954)                           0.166599   
2001: A Space Odyssey (1968)                                  0.205977   
2010: The Year We Make Contact (1984)                         0.187019   
2012 (2009)                                                   0.006377   
2046 (2004)                                                  -0.016983   
21 (2008)                                                    -0.027152   
21 Grams (2003)                                               0.035609   
21 Jump Street (2012)                                        -0.014293   
22 Jump Street (2014)                                        -0.027713   
24 Hour Party People (2002)                                   0.097485   
25th Hour (2002)                                              0.099271   
27 Dresses (2008)                                            -0.028021   
28 Days (2000)                                                0.107001   
28 Days Later (2002)                                          0.117119   
28 Weeks Later (2007)                                        -0.028967   
30 Days of Night (2007)                                       0.144642   
300 (2007)                                                    0.109059   
3000 Miles to Graceland (2001)                                0.041035   
39 Steps, The (1935)                                          0.021677   
3:10 to Yuma (2007)                                           0.131405   
40 Days and 40 Nights (2002)                                  0.115386   
40-Year-Old Virgin, The (2005)                                0.124265   
400 Blows, The (Les quatre cents coups) (1959)               -0.022021   
47 Ronin (2013)                                              -0.015018   
48 Hrs. (1982)                                                0.255706   
50 First Dates (2004)                                         0.024466   
50/50 (2011)                                                  0.027045   
6th Day, The (2000)                                           0.025169   

title                                               (500) Days of Summer (2009)  \
title                                                               

In [15]:
def get_similar(movie_name,rating):
    similar_ratings = item_similarity_df[movie_name]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [16]:
romantic_lover = [("(500) Days of Summer (2009)",5),("Alice in Wonderland (2010)",3),("Aliens (1986)",1),("2001: A Space Odyssey (1968)",2)]
similar_movies = pd.DataFrame()
for movie,rating in romantic_lover:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

similar_movies.head(10)

'burbs, The (1989)  (500) Days of Summer (2009)  \
0            0.157792                     2.500000   
1           -0.016276                     0.203998   
2           -0.304722                    -0.062634   
3           -0.102988                    -0.056808   

   *batteries not included (1987)  10 Cloverfield Lane (2016)  \
0                        0.334873                    0.356179   
1                        0.016161                    0.126834   
2                       -0.099756                   -0.214700   
3                       -0.076462                   -0.049655   

   10 Things I Hate About You (1999)  10,000 BC (2008)  101 Dalmatians (1996)  \
0                           0.684973          0.484900               0.372257   
1                           0.113241          0.092218               0.085790   
2                          -0.118754         -0.037059              -0.063992   
3                          -0.042987         -0.021729              -0.055422   

   101 Dalmatians (One Hundred and One Dalmatians) (1961)  \
0                                           0.355353        
1                                           0.072825        
2                                          -0.170195        
3                                          -0.051115        

   102 Dalmatians (2000)  12 Angry Men (1957)          ...            \
0               0.166417             0.399389          ...             
1               0.037458             0.097794          ...             
2              -0.010700            -0.280090          ...             
3              -0.014088            -0.097954          ...             

   Zero Dark Thirty (2012)  Zero Effect (1998)  Zodiac (2007)  \
0                 0.446637            0.171018       1.036463   
1                 0.085502            0.011564       0.176888   
2                -0.162387           -0.368712      -0.281119   
3                -0.048607           -0.128795      -0.175166   

   Zombieland (2009)  Zoolander (2001)  Zootopia (2016)  [REC] (2007)  \
0           0.889309          0.630565         0.540017      0.315367   
1           0.168302          0.121710         0.072590      0.050178   
2          -0.263686         -0.228562        -0.144292     -0.319902   
3          -0.098088         -0.074205        -0.049626     -0.026909   

   eXistenZ (1999)  xXx (2002)  ¡Three Amigos! (1986)  
0         0.134034    0.602730               0.314763  
1         0.025695    0.081764               0.027540  
2        -0.410899   -0.242935              -0.238410  
3        -0.153017   -0.082048              -0.091432  

[4 rows x 2949 columns]

In [17]:
similar_movies.sum().sort_values(ascending=False).head(20)

(500) Days of Summer (2009)                      2.584556
Alice in Wonderland (2010)                       1.395229
Silver Linings Playbook (2012)                   1.254800
Yes Man (2008)                                   1.116264
Adventureland (2009)                             1.112235
Marley & Me (2008)                               1.108381
About Time (2013)                                1.102192
Crazy, Stupid, Love. (2011)                      1.088757
50/50 (2011)                                     1.086517
Help, The (2011)                                 1.075963
Up in the Air (2009)                             1.053037
Holiday, The (2006)                              1.034470
Friends with Benefits (2011)                     1.030875
Notebook, The (2004)                             1.025880
Easy A (2010)                                    1.015771
Secret Life of Walter Mitty, The (2013)          0.997979
Perks of Being a Wallflower, The (2012)          0.967425
Toy Story 3 (2

In [ ]:
user = [("Grown Ups (2010)",5)]

def check_seen(movie,seen_movies):
    for item,rating in seen_movies:
        if item == movie:
            return True
    
    return False
        
similar_movies = pd.DataFrame()
for movie,rating in user:
    similar_movies = similar_movies.append(get_similar(movie,rating),ignore_index = True)

#similar_movies.head(10)
sim_movies=similar_movies.sum().sort_values(ascending=False)
for movie,score in sim_movies.iteritems():
    if not check_seen(movie,user):
        print(movie)
#similar_movies.sum().sort_values(ascending=False).head(20)